# Dependencies Imported

In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt
from scipy import stats


# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# To Reflect an existing database into a new model

Base=automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables


In [7]:
# To view classes automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Saving references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [9]:
# To create session link from Python to DB
session=Session(engine)

In [10]:
#To Inspect engine
inspector=inspect(engine)

# Exploratory Climate Analysis

# Station Table

In [11]:
stations=engine.execute('SELECT * FROM Station')

In [12]:
print(stations.keys())


RMKeyView(['id', 'station', 'name', 'latitude', 'longitude', 'elevation'])
